<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Retrieval_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Retrieval evaluation

Just to evaluate the models using precision and recall...

## Question list (30)
Have the relevant document for each so you can measure, recall, precision etc

Best to complete this after asking the question to the system?

|Doc_ID|Name|Relevant_Q|
|---|---|---|
|1|Evidence & insights - reducing household food waste & plastic packaging (packaged vs loose)|1,2,3,4,5,6,7,14,19|
|2|Food waste collection guidance |8, 17|
|3|Food Waste Reduction Roadmap - Executive Summary|9, 14, 16,19|
|4,19|Food Waste Reduction Roadmap - Hospitality|9|
|5,19|Food Waste Reduction Roadmap - Manufacturers|9|
|6,19|Food Waste Reduction Roadmap - Primary Producers|9,10|
|7,19|Food Waste Reduction Roadmap - Retail|9,11|
|8|Identifying impacts from food and farm digestates|12,13|
|9|Industry Guidance - Dealing with Household Food Waste at AD Facilities - Management of Liners|14,8, 15|
|10|Literature review - relationship between household food waste collection and food waste prevention|8, 15, 16, 17, 18, 19|
|11|The food waste reduction roadmap progress report 2022|9, 19|
|12|Towards the 2030 Food Waste Commitment -- setting our coalition baseline|14, 19|
|13|Upscaling farm food waste measurement in the UK|19, 22|
|14|WRAP-Evidence Review Plastic Packaging and Fresh Produce 171218|3, 4, 14|

1. "How should I store bananas?"
2. "What temperature should I keep milk?"
3. "How do I increase shelf life of lettuce?
4. "How does selling fresh produce loose, affect waste?"
5. "What is a best-before date for?"
6. "What is a use-by date for?"
7. "How does removing date labels help?"
8. "What are the reasons to not participate in a food waste scheme?"
9. "What is the UN's sustainable development goal with respect to food?"
10. "Which document relates to primary producers and food waste?"
11. "Which document relates to retailers and food waste?"
12. "What is the most common use for digestate?"
13. "What affects the costs of valorisation?"
14. "How many tonnes of food waste are there each year?"
15. "What is the benefit of caddy liners?"
16. "How to prevent food loss?"
17. "How to prevent food waste?"
18. "How does composting affect food waste?"
19. "What are the main drivers of food waste? got to here!"
20. "Online shopping and food waste"
21. "What is involved with Courthold 2030?"
22. "How do you measure in field food waste?"

In [1]:
!pip install -Uq sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import sqlite3
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [35]:
### I think just a dictionary will be able to store the data required

# Question
# documents have been indexed from 0 so document 1 = 0 etc
keys = list(range(1,15))
# answers (relevant questions)
values = [
    [1,2,3,4,5,6,7, 14,19],
    [8,17],
    [9,14, 16],
    [9,19],
    [9,19],
    [9,10, 19],
    [9,11, 19],
    [12,13, 19],
    [14,8,15],
    [8,15,16,17,18,19],
    [9, 19],
    [14, 19],
    [19, 22],
    [1,3,4,14],

]

QA = dict(zip(keys,values))

In [8]:
Questions = [
        "How should I store bananas?",
        "What temperature should I keep milk?",
        "How do I increase shelf life of lettuce?",
        "How does selling fresh produce loose, affect waste?",
        "What is a best-before date for?",
        "What is a use-by date for?",
        "How does removing date labels help?",
        "What are the reasons to not participate in a food waste scheme?",
        "What is the UN's sustainable development goal with respect to food?",
        "Which document relates to primary producers and food waste?",
        "Which document relates to retailers and food waste?",
        "What is the most common use for digestate?",
        "What affects the costs of valorisation?",
        "How many tonnes of food waste are there each year?",
        "What is the benefit of caddy liners?",
        "How to prevent food loss?",
        "How to prevent food waste?",
        "How does composting affect food waste?",
        "What are the main drivers of food waste?",
        "Online shopping and food waste",
        "What is involved with Courthold 2030?",
        "How do you measure in field food waste?"
]

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
def score_retrieval(question=["No question"],
                    model=None, db='/content/drive/MyDrive/Knowledge-database-6-4-24.db',
                    test=True):
    test = test
    relevant = []
    q = model.encode(question)
    conn = sqlite3.connect(db)

    # Create a cursor object
    cursor = conn.cursor()
    # load row by row
    cursor.execute('SELECT * FROM documents')

    for row in cursor:
        score  = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()
        # if meets the score threshold and is not already in the list
        if score >= 0.4 and row[1] not in relevant:
            # if high score consider it
            if test == True:
                relevant.append((row[1] + 1, row[3]))
            else:
                relevant.append((row[1] + 1))


    return list(set(relevant))

In [46]:
# What is the UN's sustainable development goal with respect to food?
retrieved_docs = score_retrieval(question = Questions[0], model=model, test=False)
retrieved_docs

[8, 1, 14]

In [48]:
Questions[0]

'How should I store bananas?'

In [47]:
## get scores for the document retrieval

def scoring(q, qa=QA):
    precision , recall, f1 = 0,0,0
    q_no = q
    # true positive
    all_relevant = []
    for k, v in QA.items():
        if q_no in v:
            all_relevant.append(k)
    correct = 0
    for i in retrieved_docs:
        if i in all_relevant:
            correct +=1
    # no relevant docs recieved - all scores are 0
    if len(all_relevant) == 0:
        return (precision, recall, f1)
    else:
        # if there are no documents that meet our criteria
        if len(retrieved_docs) == 0:
            return (precision, recall, f1)
        else:
            precision = abs(len(set(all_relevant) & set(retrieved_docs))) / abs(len(retrieved_docs))
            # recall
            recall = abs(len(set(all_relevant) & set(retrieved_docs))) / abs(len(all_relevant))
            # avoid divide by 0 error
            if precision == 0 or recall == 0:
                f1 = 0.0
            # if the other values are not zero
            else:
                f1 = (2*precision*recall)/(precision + recall)

    return (precision, recall, f1)

scoring(1)

(0.6666666666666666, 1.0, 0.8)